In [2]:
def loadtiff3d(filepath):
    """Load a tiff file into 3D numpy array"""

    import tifffile as tiff
    a = tiff.imread(filepath)

    stack = []
    for sample in a:
        stack.append(np.rot90(np.fliplr(np.flipud(sample))))
    out = np.dstack(stack)

    return out

def writetiff3d(filepath, block):
    import tifffile as tiff

    try:
        os.remove(filepath)
    except OSError:
        pass

    with tiff.TiffWriter(filepath, bigtiff=False) as tif:
        for z in range(block.shape[2]):
            saved_block = np.rot90(block[:, :, z])
            tif.save(saved_block.astype('uint8'), compress=0)

In [3]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * imageA.shape[2])
    
    return err

In [4]:
def psnr1(img1, img2):
   mse = np.mean((img1.astype("float") - img2.astype("float")) ** 2 )
   if mse < 1.0e-10:
      return 100
   return 10 * math.log10(255.0**2/mse)

def psnr2(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [65]:
from skimage import measure
import cv2
import numpy as np
import math
import os
import subprocess

path = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/checkpoints/'
methods = ['3d_unet_pixel','3d_unet_patch', 
           '3d_linknet_pixel','3d_linknet_patch',
           '3d_esp_pixel', '3d_esp_patch', 
           '3d_student_pixel', '3d_student_patch',
           '3d_vnet_pixel', '3d_vnet_patch',
           '3d_unet_pixel_epoch50', '3d_unet_pixel_epoch100',
           '3d_student_pixel_epoch50', '3d_student_pixel_epoch100',
           '3d_deeper_student_pixel', '3d_deeper_student_patch',
           '3d_more_inception_student_pixel', '3d_more_inception_student_v2_pixel',
          '3d_more_inception_student_v2_patch', '3d_more_inception_student_v2_patch_less']
chosen_method_lst = [2,3,4,5,6,7,8,9,14,15,17,18]

for c in chosen_method_lst:
    chosen_method = methods[c]
    prefix = path+chosen_method+'/test_full/'
    if not os.path.exists(prefix+'real_B'):
        os.makedirs(prefix+'real_B')

    if not os.path.exists(prefix+'fake_B'):
        os.makedirs(prefix+'fake_B')

    for fname in os.listdir(prefix):
        if 'fake' in fname and fname.endswith('.tif'):
            subprocess.call(['cp', prefix+fname, prefix+'fake_B/'+fname])
        if 'real' in fname and fname.endswith('.tif'):
            subprocess.call(['cp', prefix+fname, prefix+'real_B/'+fname])


    real_path = path+chosen_method+'/test_full/real_B/'
    fake_path = path+chosen_method+'/test_full/fake_B/'

    mse_lst = []
    ssim_lst = []
    psnr_lst = []
    for _,(real,fake) in enumerate(zip(os.listdir(real_path), os.listdir(fake_path))):
        a = loadtiff3d(real_path+real)
        b = loadtiff3d(fake_path+fake)

        s = measure.compare_ssim(a, b)        
        m = mse(a,b)    
        psnr = psnr1(a,b)

        ssim_lst.append(s)
        mse_lst.append(m)
        psnr_lst.append(psnr)

    #     print(a.shape)

    print(chosen_method)
    avg_ssim = np.mean(ssim_lst)
    avg_mse = np.mean(mse_lst)
    avg_psnr = np.mean(psnr_lst)
    std_ssim = np.std(ssim_lst)
    std_mse = np.std(mse_lst)
    std_psnr = np.std(psnr_lst)

    print('average SSIM:', avg_ssim, '+/-', std_ssim)
    print('average mse:', avg_mse, '+/-', std_mse)
    print('mse details: ', mse_lst)
    print('average psnr1:', avg_psnr, '+/-', std_psnr)

3d_linknet_pixel
average SSIM: 0.8260449438445416 +/- 0.09684097237086776
average mse: 69.10944564074279 +/- 48.4989924434838
mse details:  [41.05104977627869, 49.15526479150796, 35.505406027145156, 54.621049926113926, 165.2144576826682]
average psnr1: 30.509609840719826 +/- 2.36842123675203
3d_linknet_patch
average SSIM: 0.7399852867838238 +/- 0.08823236169744723
average mse: 110.68063490085288 +/- 65.91019095153827
mse details:  [62.65352858102642, 75.43651108363584, 75.57037159685083, 99.40083611724515, 240.34192712550606]
average psnr1: 28.26860708141851 +/- 2.074146500395217
3d_esp_pixel
average SSIM: 0.822267569638108 +/- 0.08571984284701989
average mse: 70.37565249973356 +/- 41.90296102293811
mse details:  [54.978941220023856, 43.39698985268248, 45.151654781599206, 54.71173467405195, 153.63894197031038]
average psnr1: 30.216980723709145 +/- 2.0197106608272186
3d_esp_patch
average SSIM: 0.8069559485259402 +/- 0.12841421968228361
average mse: 95.88995221066105 +/- 61.9602433806734

In [ ]:
0.962304583822089
47.94498018729519
31.32337217371245
0.9271199179527903
61.44465627917934
30.245962417394615
0.9073358005950121
72.50591753239678
29.527069081342834
0.7561640260977293
81.96205886238761
28.994675016543287
0.6233789221225888
210.25498823983034
24.903340527626014
3d_more_inception_student_v2_patch
average SSIM: 0.835260650118042 +/- 0.21188172799545313
average mse: 94.82252022021785 +/- 115.43246801961249
average psnr1: 28.998883843323835 +/- 4.095543315697821

## U-net
0.9536027670736983
36.8797522840598
0.9529356751639284
29.883761368658355
0.925281883330857
35.04958424670749
0.8269231907544411
42.290504667978965
0.6710535649510824
190.48211760169656
average SSIM: 0.8659594162548014
average mse: 66.91714403382022
average psnr1: 31.144797200405254
average psnr2: 31.144797200405254
    
Link-net
0.949205702591625
38.187937239201275
0.9298947389583114
34.44531591500397
0.8786159546436956
44.692158591472094
0.8403984850399471
46.61583321398317
0.7149634649807374
212.76616464237517
average SSIM: 0.8626156692428634
average mse: 75.34148192040713
average psnr1: 30.599356169078483
average psnr2: 30.599356169078483

In [11]:
from skimage import measure
import cv2
import numpy as np
import math
import os
import subprocess

result_prefix = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/checkpoints/'
real_path = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/datasets/datasets/fly/fly3d/tokyo_gt/'
methods = ['more_inception_tokyo', 'more_inception_tokyo_vnet',
          'more_inception_tokyo_esp', 'more_inception_tokyo_link',
          'more_inception_tokyo_student', 'more_inception_tokyo_student_deeper',
          ]
chosen_method_lst = [5]

for c in chosen_method_lst:
    chosen_method = methods[c]
    fake_path = result_prefix+chosen_method+'/test_full/'
#     if not os.path.exists(prefix+'real_B'):
#         os.makedirs(prefix+'real_B')

#     if not os.path.exists(prefix+'fake_B'):
#         os.makedirs(prefix+'fake_B')

#     for fname in os.listdir(prefix):
#         if 'fake' in fname and fname.endswith('.tif'):
#             subprocess.call(['cp', prefix+fname, prefix+'fake_B/'+fname])
#         if 'real' in fname and fname.endswith('.tif'):
#             subprocess.call(['cp', prefix+fname, prefix+'real_B/'+fname])


#     real_path = path+chosen_method+'/test_full/real_B/'
#     fake_path = path+chosen_method+'/test_full/fake_B/'

    mse_lst = []
    ssim_lst = []
    psnr_lst = []
    for _,(real,fake) in enumerate(zip(os.listdir(real_path), os.listdir(fake_path))):
        a = loadtiff3d(real_path+real)
        b = loadtiff3d(fake_path+fake)

        s = measure.compare_ssim(a, b)        
        m = mse(a,b)    
        psnr = psnr1(a,b)

        ssim_lst.append(s)
        mse_lst.append(m)
        psnr_lst.append(psnr)

    #     print(a.shape)

    print(chosen_method)
    avg_ssim = np.mean(ssim_lst)
    avg_mse = np.mean(mse_lst)
    avg_psnr = np.mean(psnr_lst)
    std_ssim = np.std(ssim_lst)
    std_mse = np.std(mse_lst)
    std_psnr = np.std(psnr_lst)

    print('average SSIM:', avg_ssim, '+/-', std_ssim)
    print('average mse:', avg_mse, '+/-', std_mse)
    print('mse details: ', mse_lst)
    print('average psnr1:', avg_psnr, '+/-', std_psnr)
    print()

more_inception_tokyo_student_deeper
average SSIM: 0.9951114052702601 +/- 0.0017509732805668279
average mse: 5.554155924383903 +/- 2.009712119095328
mse details:  [4.835467180391992, 8.189093926743745, 8.348764637421219, 4.003460336924502, 4.85907112008943, 3.089078344732537]
average psnr1: 40.96727991016743 +/- 1.5657544543756912

